In [1]:
import os
import sys
import re
import pandas as pd
import numpy as np
from enum import Enum
import zipfile
from typing import Optional, Union, List, Callable, Dict, Tuple
from dataclasses import dataclass, make_dataclass, field
from abc import ABC
from importnb import Notebook

PKG_ROOT = os.path.dirname(os.path.realpath(os.getcwd()))
if not PKG_ROOT in sys.path:
    sys.path.append(PKG_ROOT)

In [2]:
with Notebook():
    from src.basestructs import *
    from src.baseblocks import *
    from src.basesegs import *

In [ ]:
class Party(SegBlock):
    
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.CREIndexLength, self.CREOffsetLoc, self.CRESizeLoc = self.get_CRE_locs()
        self.parse_party()
        
    def get_CRE_locs(self):
        loc_df = self.savRef.creLocs
        CREIndexLen = self.to_int(loc_df[loc_df['SubSegs']=='Header']['SizeValue'].iloc[0])
        CREOffsetLoc, CRESizeLoc =  loc_df[loc_df['SubSegs']=='CRE'][['OffsetLoc', 'SizeLoc']].iloc[0].apply(self.to_int)
        return CREIndexLen, CREOffsetLoc, CRESizeLoc
    
    def parse_party(self):
        self.indexHeader = self.buffer[: self.CREIndexLength * self.countBlock.value]
        self.CREOffsetBlocks = [ValueBlock(self.buffer, 'CREOFFSET', self.CREOffsetLoc + self.CREIndexLength * i, 4) for i in range(self.countBlock.value)]
        self.CRESizeBlocks = [ValueBlock(self.buffer, 'CRESIZE', self.CRESizeLoc + self.CREIndexLength * i, 4) for i in range(self.countBlock.value)]
        vRecords = self.savRef.creValues
        tRecords = self.savRef.creTables
        sRecords = self.savRef.creSegs
        self.CRES = [CRE(self.savRef
                        ,self
                        ,self.pbuffer[o.value: o.value + s.value]
                        ,vRecords
                        ,tRecords
                        ,sRecords
                        ) for o, s in zip(self.CREOffsetBlocks, self.CRESizeBlocks)]
        
    def pack(self):
        [_.pack() for _ in self.CRES]
        self.buffer = bytearray(b''.join([self.indexHeader, *[_.buffer for _ in self.CRES]]))
        self.sizeValue = len(self.buffer)
        self.offsetValue = self.previous.offsetValue + self.previous.sizeValue
        creSizes = np.array([_.size for _ in self.CRES])
        creOffsets = np.concatenate([[0], creSizes]).cumsum()[:-1] + len(self.indexHeader) + self.offsetValue
        for o, s, ob, sb in zip(creOffsets, creSizes, self.CREOffsetBlocks, self.CRESizeBlocks):
            ob.bind_buffer(self.buffer).set_value(int(o))
            sb.bind_buffer(self.buffer).set_value(int(s))
        return self.buffer

In [3]:
class GamCreBase(ABC, BlockBase):
    
    SEGMAP = {"KnownSpells": KnownSpells
             ,"SpellMemorization": SpellMemorization
             ,"MemorizedSpells": MemorizedSpells
             ,"Effects": Effects
             ,"Items": Items
             ,"ItemSlots": ItemSlots
             ,"Party": Party
             ,"NPC": Party
             ,"Global": Globals}
        
        
    def init_seg_size(self):
        df = self.segRecords.copy()
        df.insert(df.shape[1]
                 ,'OffsetValue'
                 ,self.segRecords['OffsetLoc'].apply(lambda x: np.nan if pd.isnull(x) else self.bytes2Num(self.buffer[self.to_int(x)
                                                                                                                     :self.to_int(x)+4])))
        df1 = df[df['OffsetValue']!=0].fillna(value={'OffsetValue': 0})
        df2 = df[df['OffsetValue']==0]
        df1['SizeValue'] = df1['OffsetValue'].shift(-1, fill_value=self.size) - df1['OffsetValue']
        self.segRecords = pd.concat([df1, df2]).fillna(value={'SizeValue': 0}).sort_index()
        
    
    def init_values(self, buffer:Optional[bytearray]=None): 
        self.VALUES = pd.Series([], dtype=object)
        if not self.valueRecords.empty:
            if buffer == None:
                buffer = self.SEGS[0].buffer
            self.VALUES = self.valueRecords.apply(lambda x: ValueBlock(buffer, *x), axis=1)
            for i in range(self.VALUES.size):
                setattr(self, self.VALUES[i].name, self.VALUES[i])
            
            # self.valueRecords.apply(lambda x: setattr(self, x[0], ValueBlock(buffer, *x[1:])),axis=1)
            # self.VALUES = make_dataclass('VALUES', self.valueRecords.apply(lambda x: (x[0]
            #                                                                          ,ValueBlock
            #                                                                          ,ValueBlock(buffer, *x[1:]))
            #                                                                ,axis=1))
            
            
    def init_tables(self, buffer:Optional[bytearray]=None):
        self.TABLEVALUES = pd.Series([], dtype=object)
        if not self.tableRecords.empty:
            if buffer == None:
                buffer = self.buffer
            self.TABLEVALUES = self.tableRecords.apply(lambda x: TableBlock(buffer, *x, getattr(self.savRef, x['RefTable'])) ,axis=1)
            for i in range(self.TABLEVALUES.size):
                setattr(self, self.TABLEVALUES[i].name, self.TABLEVALUES[i])
            # self.TABLES = make_dataclass('TABLES', self.tableRecords.apply(lambda x: (x[0]
            #                                                                          ,TableBlock
            #                                                                          ,TableBlock(buffer
            #                                                                                      ,*x[1:]
            #                                                                                      ,getattr(self.savRef, x['RefTable'])))
            #                                                                ,axis=1))
                                                                           
            
    def init_segs(self, buffer:Optional[bytearray]=None):
        '''
        Need to keep sequence order
        So use pd.Series instead of dataclass
        '''
        self.SEGS = pd.Series([], dtype=object)
        if not self.segRecords.empty:
            if buffer == None:
                buffer = self.buffer
            self.SEGS = (self.segRecords
                         .set_index(self.segRecords.columns[0])
                         .apply(lambda x: self.SEGMAP.get(x.name, SegBlock)(self.savRef, self, buffer, x.name, *x), axis=1))
            for i in range(1, self.SEGS.size):
                self.SEGS[i].previous = self.SEGS[i-1]
                setattr(self, self.SEGS[i].name, self.SEGS[i])
            
            
    def pack(self):
        self.buffer = bytearray(b''.join([_.pack() for _ in self.SEGS]))
        self.size = len(self.buffer)
        for seg in self.SEGS:
            seg.offsetBlock.bind_buffer(self.buffer).set_value(seg.offsetValue)
            seg.countBlock.bind_buffer(self.buffer).set_value(seg.countValue)
            seg.sizeBlock.bind_buffer(self.buffer).set_value(seg.sizeValue)
        return self.buffer
    

In [4]:
class CRE(GamCreBase):
    
    def __init__(self
                ,savRef:object
                ,parentRef:object
                ,buffer: bytearray
                ,valueRecords: Optional[pd.DataFrame] = None
                ,tableRecords: Optional[pd.DataFrame] = None
                ,segRecords: Optional[pd.DataFrame] = None
                ):
        self.savRef = savRef
        self.parentRef = parentRef
        self.buffer = buffer
        self.size = len(self.buffer)
        if valueRecords is None:
            self.valueRecords = pd.read_csv(os.path.join(self.savRef.resourceDir, 'CREVALUES.csv'), index_col=0)
        else:
            self.valueRecords = valueRecords.copy()
        if tableRecords is None:
            self.tableRecords = pd.read_csv(os.path.join(self.savRef.resourceDir, 'CRETABLES.csv'), index_col=0)
        else:
            self.tableRecords = tableRecords.copy()
        if segRecords is None:
            self.segRecords = pd.read_csv(os.path.join(self.savRef.resourceDir, 'CRESEGS.csv'), index_col=0)
        else:
            self.segRecords = segRecords.copy()
        self.VALUES = make_dataclass('SEGS', [])
        self.TABLES = make_dataclass('SEGS', [])
        self.SEGS = pd.Series([], dtype=object)
        self.init_seg_size()
        self.init_segs() 
        self.init_values()
        self.init_tables()

In [5]:
class GAM(GamCreBase):
    
    def __init__(self
                 ,savRef: object
                 ,buffer: bytearray):
        self.savRef = savRef
        self.resourceDir = savRef.resourceDir
        self.buffer = buffer
        self.size = len(self.buffer)
        self.valueRecords = savRef.gamValues
        self.segRecords = savRef.gamSegs
        self.VALUES = make_dataclass('SEGS', [])
        self.SEGS = pd.Series([], dtype=object)
        self.init_seg_size()
        self.init_segs()
        self.init_values()
        


In [6]:
class Sav(GamCreBase):
    
    def __init__(self, savefile:str, game:str='BGEE'):   # for other games, RACE, KIT, STATES, CLASS may need to copy from BGEE since the list is more comprehensive
        self.savefile = os.path.join(PKG_ROOT, 'saves', 'original', savefile)
        self.modified = os.path.join(PKG_ROOT, 'saves', 'modified', 'Edited_'+savefile)
        self.resourceDir = os.path.join(PKG_ROOT, 'resources', game)
        self.zipfile = zipfile.ZipFile(self.savefile)
        self.filelist = self.zipfile.namelist()
        self.files = [self.zipfile.read(_) for _ in self.filelist]
        self.gamStr = self.files[1].decode('latin')
        self.gamBuffer = bytearray(self.files[1])
        self.gamVersion = re.findall(r'GAME\s*V\d+\.\d+', self.gamStr)[0]  #always 'GAMEV2.0'
        self.creVersion = re.findall(r'CRE\s*V\d+\.\d+', self.gamStr)[0]  #always 'CRE V1.0'
        self.load_dfs()
        self.make_names()
        self.GAM = GAM(savRef=self, buffer=self.gamBuffer)
        self.Party = self.GAM.SEGS.Party.CRES
        self.NPC = self.GAM.SEGS.NPC.CRES
        
        
    def load_dfs(self):
        self.gamValues = pd.read_csv(os.path.join(self.resourceDir, 'GAMVALUES.csv'), index_col=0)
        self.gamSegs = pd.read_csv(os.path.join(self.resourceDir, 'GAMSEGS.csv'), index_col=0)
        self.creValues = pd.read_csv(os.path.join(self.resourceDir, 'CREVALUES.csv'), index_col=0)
        self.creTables = pd.read_csv(os.path.join(self.resourceDir, 'CRETABLES.csv'), index_col=0)
        self.creSegs = pd.read_csv(os.path.join(self.resourceDir, 'CRESEGS.csv'), index_col=0)
        self.creLocs = pd.read_csv(os.path.join(self.resourceDir, 'CRELOC.csv'), index_col=0)
    
    def make_names(self):
        for itm in ['ITEM', 'SPELL', 'EFFECT']:
            f = pd.read_csv(os.path.join(self.resourceDir, itm + '.csv'), index_col=0)
            setattr(self, itm+'CODES', IDS.make_names(itm+'CODES', f))
            setattr(self, itm, IDS.make_enum(itm, f, nameCol=-1, valueCol=-2))
            if itm == 'ITEM':
                setattr(self, itm+'CONTAINER', IDS.make_enum(itm+'CONTAINER', f[['ItemSlot', 'Item']]))
            #setattr(self, itm, IDS.make_enum(itm, IDS.clean_duplicates(f), nameCol=-1, valueCol=-2))
        for itm in ['WEAPON', *self.creTables['RefTable'].drop_duplicates()]: #Race, Kit, Class, Gender, Alignment
            setattr(self, itm, IDS.make_enum(itm, pd.read_csv(os.path.join(self.resourceDir, itm + '.csv'), index_col=0)))
        for itm in ['PARTYSLOTS', 'NPCSLOTS']:
            f = pd.read_csv(os.path.join(self.resourceDir, itm + '.csv'), index_col=0)
            f = f.groupby(f.columns[0]).apply(lambda x: x.index.to_list()).to_frame().reset_index()
            setattr(self, itm, IDS.make_enum(itm, f, nameCol=0))
               
            
    def pack(self, filename:str=None):
        if filename is None:
            filename = self.modified
        self.zipfile.close()
        with zipfile.ZipFile(filename, 'w') as target:
            target.writestr(self.filelist[0], self.files[0], compress_type=zipfile.ZIP_STORED)
            target.writestr(self.filelist[1], self.GAM.pack(), compress_type=zipfile.ZIP_STORED)
            for _name, _f in zip(self.filelist[2:], self.files[2:]):
                target.writestr(_name, _f, compress_type=zipfile.ZIP_STORED)
                
                
    def optimize(self):
        [_.set_value(_.optimize) for _ in self.GAM.VALUES]
        [_.set_value(_.optimize) for c in self.Party for _ in c.VALUES]
            
        